<h1>Snow Clearance Fines, 2019-2023</h1>
15 February 2024

This analysis looks at fines levied for uncleared sidewalks, based on FOIA data requested from the Department of Administrative Hearings (H064920-011124.xlsx). This dataset contained 3058 records dating from 1/1/2001 to 9/12/2023; filtered for those between 7/1/2019 and 6/30/2023 has 2560 records. Four of these could not be geocoded due to "unknown" address.<br>
<br>
<ol>
<li><a href="#read">Read Data</a>
             <li><a href="#community">Summarize Dockets by Community</a>- look for patterns across Chicago community areas
</ol>

### Record Count
<ul>
    <li>2556 valid fines records from July 1 2019 to June 30 2023
        <li>1912 dockets. Some dockets contain multiple fines records, which are identical except for a different fine amount in each record.
            <li>1735 addresses. some addresses have been fined by both CDOT and Streets & Sanitation, with a separate court docket for each.
                <li>some addresses have been fined by multiple agencies
    </ul>

### Preliminary Findings    
<ul>
    <li>73% of court dockets were issued by CDOT, 26% by Streets and Sanitation. The remaining 1% were issued by the police or Business Affairs and Consumer Protection
    <li>Englewood, Garfield Ridge, and West Englewood are the three communities with the highest number of dockets per capita
    <li>Only 25 court dockets were issued by police. West Englewood, Englewood, and Garfield Ridge have the highest rates and account for half the dockets citywide.
        <li>For dockets issued by CDOT, Garfield Ridge, Grand Boulevard, and Armour Square have the highest per capita rate
            <li>For dockets issued by Streets and Sanitation, Englewood, West Englewood, and Brighton Park have the highest per capita rate
    </ul>

<a name="read"></a>
# 1. Read and Prepare Geocoded Fines Data

In [1]:
import pandas as pd

Note the following data preparation steps prior to this notebook
<ol>
<li>Prepared data by parsing dates and correcting data entry errors in addresses; see <a href="fines-01-prep-data.ipynb">fines-01-prep-data.ipynb</a>.
    <li>Geocoded addresses to identify lat and long coordinates, and spatially joined addresses to Community Areas shapefile. I did this offline in QGIS.
        </ol>

In [6]:
df_dockets = pd.read_csv("../../data/05-standardized/fines-by-docket.csv")
df_dockets.head()

,docket,dept,address,lat,long,community,violation_date,season,n_records,total_fine
0,19DS68300L,STRTSAN,4710 S WESTERN AVE,41.807859,-87.684797,BRIGHTON PARK,2019/11/13,2019-2020,1,150.0
1,19DS69216L,STRTSAN,1425 W MORSE AVE,42.007451,-87.666828,ROGERS PARK,2019/11/13,2019-2020,1,50.0
2,19DS70010L,STRTSAN,715 E 47TH ST,41.809338,-87.608013,GRAND BOULEVARD,2019/11/13,2019-2020,1,150.0
3,19DS72153L,STRTSAN,300 W WASHINGTON ST,41.882868,-88.210529,LOOP,2019/11/12,2019-2020,3,650.0
4,19DS72160L,STRTSAN,6929 N SHERIDAN RD,41.959813,-87.654693,UPTOWN,2019/11/14,2019-2020,1,500.0


In [7]:
len(df_dockets)

1912

<a name="community"></a>
# Create Community-Level Dataframe
Note that 7 communities have no dockets

In [8]:
df_community = df_dockets.pivot_table(index='community',
                             columns='dept',
                             values='total_fine',
                             aggfunc=['count'],
                             fill_value=0).reset_index()
# Flatten the MultiIndex in columns
df_community.columns = ['_'.join(col).strip() for col in df_community.columns.values]

df_community = df_community.rename(columns={'community_':'community'})

# Reset the index to flatten it
df_community.reset_index(drop=True, inplace=True)

df_community['n_dockets']=df_community['count_BAFCONP']+df_community['count_POLICE']+df_community['count_STRTSAN']+df_community['count_TRANPORT']

df_community.head()

,community,count_BAFCONP,count_POLICE,count_STRTSAN,count_TRANPORT,n_dockets
0,ALBANY PARK,0,0,0,15,15
1,ARCHER HEIGHTS,0,0,0,12,12
2,ARMOUR SQUARE,0,0,0,26,26
3,ASHBURN,0,0,0,2,2
4,AUBURN GRESHAM,0,0,3,16,19


In [57]:
df_community[df_community['community']=='LOOP']

,community,count_BAFCONP,count_POLICE,count_STRTSAN,count_TRANPORT,n_dockets
38,LOOP,0,0,3,21,24


# Merge Community-Level Datasets

### read community population

In [14]:
# retrieved on 1/11/24, but 2020 Census Population figures should be static
df_population = pd.read_csv("../../data/02-tidied/census-by-community.csv")

# simplify dataframe to get only essentials
df_population = df_population[['GEOID','GEOG','2020_POP']]
df_population = df_population.rename(columns={'GEOG':'COMMUNITY_NAME'})
df_population['COMMUNITY_CAPS']=df_population['COMMUNITY_NAME'].str.upper()
df_population.head()

,GEOID,COMMUNITY_NAME,2020_POP,COMMUNITY_CAPS
0,14,Albany Park,48396,ALBANY PARK
1,57,Archer Heights,14196,ARCHER HEIGHTS
2,34,Armour Square,13890,ARMOUR SQUARE
3,70,Ashburn,41098,ASHBURN
4,71,Auburn Gresham,44878,AUBURN GRESHAM


### merge in community population data

In [15]:
df_community_summary = pd.merge(df_community,df_population,left_on='community',right_on='COMMUNITY_CAPS',how='right')
df_community_summary.head()

,community,count_BAFCONP,count_POLICE,count_STRTSAN,count_TRANPORT,n_dockets,GEOID,COMMUNITY_NAME,2020_POP,COMMUNITY_CAPS
0,ALBANY PARK,0.0,0.0,0.0,15.0,15.0,14,Albany Park,48396,ALBANY PARK
1,ARCHER HEIGHTS,0.0,0.0,0.0,12.0,12.0,57,Archer Heights,14196,ARCHER HEIGHTS
2,ARMOUR SQUARE,0.0,0.0,0.0,26.0,26.0,34,Armour Square,13890,ARMOUR SQUARE
3,ASHBURN,0.0,0.0,0.0,2.0,2.0,70,Ashburn,41098,ASHBURN
4,AUBURN GRESHAM,0.0,0.0,3.0,16.0,19.0,71,Auburn Gresham,44878,AUBURN GRESHAM


### calculate dockets per capita

In [18]:
# per 10,000 capita, per year over 4 years
df_community_summary['dp10k'] = \
(10000/4)*df_community_summary['n_dockets']/df_community_summary['2020_POP']
df_community_summary['streets_p10k'] = \
(10000/4)*df_community_summary['count_STRTSAN']/df_community_summary['2020_POP']
df_community_summary['cdot_p10k'] = \
(10000/4)*df_community_summary['count_TRANPORT']/df_community_summary['2020_POP']
df_community_summary['police_p10k'] = \
(10000/4)*df_community_summary['count_POLICE']/df_community_summary['2020_POP']
df_community_summary['business_p10k'] = \
(10000/4)*df_community_summary['count_BAFCONP']/df_community_summary['2020_POP']

### merge 311 unshoveled sidewalk complaints

In [27]:
df_311_unshoveled = pd.read_csv("../../results/311-uncleared-by-community.csv")

In [28]:
df_311_unshoveled.head()

,COMMUNITY_NAME,Sidewalks Per 10k,Snow – Uncleared Sidewalk Complaint,2020_POP,COMMUNITY_CAPS
0,Lincoln Square,57.354176,929,40494,LINCOLN SQUARE
1,Logan Square,55.361753,1587,71665,LOGAN SQUARE
2,Uptown,46.124655,1055,57182,UPTOWN
3,West Town,45.824267,1609,87781,WEST TOWN
4,Lincoln Park,42.628951,1202,70492,LINCOLN PARK


In [29]:
# take only the fields I need
merge_cols = ['COMMUNITY_NAME','Sidewalks Per 10k','Snow – Uncleared Sidewalk Complaint']

In [43]:
df_community_summary = pd.merge(df_community_summary,df_311_unshoveled[merge_cols],on="COMMUNITY_NAME")
df_community_summary = df_community_summary.rename(columns={'Snow – Uncleared Sidewalk Complaint':'snow_complaints'})
df_community_summary.head()

,community,count_BAFCONP,count_POLICE,count_STRTSAN,count_TRANPORT,n_dockets,GEOID,COMMUNITY_NAME,2020_POP,COMMUNITY_CAPS,dp10k,streets_p10k,cdot_p10k,police_p10k,business_p10k,Sidewalks Per 10k_x,snow_complaints,citation_rate,Sidewalks Per 10k_y,snow_complaints
0,ALBANY PARK,0.0,0.0,0.0,15.0,15.0,14,Albany Park,48396,ALBANY PARK,0.774857,0.00000,0.774857,0.0,0.0,20.662865,400,0.037500,20.662865,400
1,ARCHER HEIGHTS,0.0,0.0,0.0,12.0,12.0,57,Archer Heights,14196,ARCHER HEIGHTS,2.113271,0.00000,2.113271,0.0,0.0,10.566357,60,0.200000,10.566357,60
2,ARMOUR SQUARE,0.0,0.0,0.0,26.0,26.0,34,Armour Square,13890,ARMOUR SQUARE,4.679626,0.00000,4.679626,0.0,0.0,13.858891,77,0.337662,13.858891,77
3,ASHBURN,0.0,0.0,0.0,2.0,2.0,70,Ashburn,41098,ASHBURN,0.121660,0.00000,0.121660,0.0,0.0,6.569663,108,0.018519,6.569663,108
4,AUBURN GRESHAM,0.0,0.0,3.0,16.0,19.0,71,Auburn Gresham,44878,AUBURN GRESHAM,1.058425,0.16712,0.891305,0.0,0.0,7.130443,128,0.125000,7.130443,128


In [44]:
df_community_summary['citation_rate'] = df_community_summary['count_TRANPORT']/df_community_summary['snow_complaints']

ValueError: cannot reindex on an axis with duplicate labels

# Summarize

### top communities for dockets per capita

In [45]:
df_community_summary.sort_values(by='dp10k',ascending = False).head()

,community,count_BAFCONP,count_POLICE,count_STRTSAN,count_TRANPORT,n_dockets,GEOID,COMMUNITY_NAME,2020_POP,COMMUNITY_CAPS,dp10k,streets_p10k,cdot_p10k,police_p10k,business_p10k,Sidewalks Per 10k_x,snow_complaints,citation_rate,Sidewalks Per 10k_y,snow_complaints
23,ENGLEWOOD,0.0,4.0,58.0,7.0,69.0,68,Englewood,24369,ENGLEWOOD,7.078666,5.950183,0.718125,0.410357,0.0,9.540810,93,0.075269,9.540810,93
27,GARFIELD RIDGE,0.0,3.0,2.0,86.0,91.0,56,Garfield Ridge,35439,GARFIELD RIDGE,6.419481,0.141088,6.066763,0.211631,0.0,11.428088,162,0.530864,11.428088,162
69,WEST ENGLEWOOD,0.0,6.0,60.0,9.0,75.0,67,West Englewood,29647,WEST ENGLEWOOD,6.324417,5.059534,0.758930,0.505953,0.0,5.059534,60,0.150000,5.059534,60
28,GRAND BOULEVARD,0.0,0.0,5.0,47.0,52.0,38,Grand Boulevard,24589,GRAND BOULEVARD,5.286917,0.508357,4.778560,0.000000,0.0,21.351011,210,0.223810,21.351011,210
54,OAKLAND,0.0,0.0,1.0,12.0,13.0,36,Oakland,6799,OAKLAND,4.780115,0.367701,4.412414,0.000000,0.0,15.443448,42,0.285714,15.443448,42


### top communities for Streets and Sanitation dockets per capita

In [46]:
df_community_summary.sort_values(by='streets_p10k',ascending = False).head(10)

,community,count_BAFCONP,count_POLICE,count_STRTSAN,count_TRANPORT,n_dockets,GEOID,COMMUNITY_NAME,2020_POP,COMMUNITY_CAPS,dp10k,streets_p10k,cdot_p10k,police_p10k,business_p10k,Sidewalks Per 10k_x,snow_complaints,citation_rate,Sidewalks Per 10k_y,snow_complaints
23,ENGLEWOOD,0.0,4.0,58.0,7.0,69.0,68,Englewood,24369,ENGLEWOOD,7.078666,5.950183,0.718125,0.410357,0.0,9.540810,93,0.075269,9.540810,93
69,WEST ENGLEWOOD,0.0,6.0,60.0,9.0,75.0,67,West Englewood,29647,WEST ENGLEWOOD,6.324417,5.059534,0.758930,0.505953,0.0,5.059534,60,0.150000,5.059534,60
11,BRIGHTON PARK,0.0,0.0,70.0,11.0,81.0,58,Brighton Park,45053,BRIGHTON PARK,4.494706,3.884314,0.610392,0.000000,0.0,6.880785,124,0.088710,6.880785,124
49,NEW CITY,0.0,0.0,62.0,7.0,69.0,61,New City,43628,NEW CITY,3.953883,3.552764,0.401119,0.000000,0.0,7.048226,123,0.056911,7.048226,123
29,GREATER GRAND CROSSING,0.0,1.0,34.0,1.0,36.0,69,Greater Grand Crossing,31471,GREATER GRAND CROSSING,2.859776,2.700899,0.079438,0.079438,0.0,10.565282,133,0.007519,10.565282,133
36,KENWOOD,0.0,0.0,15.0,18.0,33.0,39,Kenwood,19116,KENWOOD,4.315756,1.961707,2.354049,0.000000,0.0,15.955221,122,0.147541,15.955221,122
59,ROGERS PARK,0.0,0.0,32.0,7.0,39.0,1,Rogers Park,55628,ROGERS PARK,1.752714,1.438125,0.314590,0.000000,0.0,22.965054,511,0.013699,22.965054,511
19,EAST GARFIELD PARK,0.0,0.0,9.0,29.0,38.0,27,East Garfield Park,19992,EAST GARFIELD PARK,4.751901,1.125450,3.626451,0.000000,0.0,18.882553,151,0.192053,18.882553,151
41,LOWER WEST SIDE,0.0,0.0,15.0,23.0,38.0,31,Lower West Side,33751,LOWER WEST SIDE,2.814731,1.111078,1.703653,0.000000,0.0,24.073361,325,0.070769,24.073361,325
39,LINCOLN SQUARE,0.0,0.0,13.0,24.0,37.0,4,Lincoln Square,40494,LINCOLN SQUARE,2.284289,0.802588,1.481701,0.000000,0.0,57.354176,929,0.025834,57.354176,929


### top communities for CDOT dockets per capita

In [47]:
df_top = df_community_summary.sort_values(by='cdot_p10k',ascending = False).head(10)
df_top = df_top[['COMMUNITY_NAME','count_TRANPORT','snow_complaints','cdot_p10k']]
df_top

,COMMUNITY_NAME,count_TRANPORT,snow_complaints,snow_complaints,cdot_p10k
27,Garfield Ridge,86.0,162,162,6.066763
28,Grand Boulevard,47.0,210,210,4.778560
2,Armour Square,26.0,77,77,4.679626
38,Lincoln Park,126.0,1202,1202,4.468592
54,Oakland,12.0,42,42,4.412414
10,Bridgeport,53.0,307,307,3.931517
19,East Garfield Park,29.0,151,151,3.626451
47,Near South Side,41.0,174,174,3.559646
43,Montclare,19.0,81,81,3.298382
48,Near West Side,83.0,590,590,3.056820


In [48]:
#calculate citywide average dockets per capita
(10000/4)*df_community_summary['count_TRANPORT'].sum()/df_community_summary['2020_POP'].sum()

1.2638262779715173

### top communities for CDOT dockets per complaint

In [49]:
df_per_complaint = df_community_summary.sort_values(by='citation_rate',ascending = False)
df_per_complaint = df_per_complaint.dropna()
df_top_communities = df_per_complaint[['COMMUNITY_NAME','count_TRANPORT','snow_complaints','citation_rate']].head()
df_top_communities

,COMMUNITY_NAME,count_TRANPORT,snow_complaints,snow_complaints,citation_rate
27,Garfield Ridge,86.0,162,162,0.530864
16,Clearing,22.0,54,54,0.407407
2,Armour Square,26.0,77,77,0.337662
54,Oakland,12.0,42,42,0.285714
47,Near South Side,41.0,174,174,0.235632


In [50]:
df_bottom_communities = df_per_complaint[['COMMUNITY_NAME','count_TRANPORT','snow_complaints','citation_rate']].tail(20)
df_bottom_communities

,COMMUNITY_NAME,count_TRANPORT,snow_complaints,snow_complaints,citation_rate
24,Forest Glen,5.0,172,172,0.029070
53,Norwood Park,5.0,175,175,0.028571
18,Dunning,5.0,183,183,0.027322
33,Hyde Park,5.0,192,192,0.026042
39,Lincoln Square,24.0,929,929,0.025834
7,Avondale,13.0,509,509,0.025540
56,Portage Park,14.0,551,551,0.025408
37,Lake View,42.0,1665,1665,0.025225
34,Irving Park,16.0,752,752,0.021277
73,West Ridge,17.0,804,804,0.021144


In [51]:
df_community_summary['count_TRANPORT'].sum()/df_community_summary['snow_complaints'].sum()

snow_complaints    0.0654
snow_complaints    0.0654
dtype: float64

### top communities for police dockets per capita

In [52]:
df_community_summary.sort_values(by='police_p10k',ascending = False).head()

,community,count_BAFCONP,count_POLICE,count_STRTSAN,count_TRANPORT,n_dockets,GEOID,COMMUNITY_NAME,2020_POP,COMMUNITY_CAPS,dp10k,streets_p10k,cdot_p10k,police_p10k,business_p10k,Sidewalks Per 10k_x,snow_complaints,citation_rate,Sidewalks Per 10k_y,snow_complaints
69,WEST ENGLEWOOD,0.0,6.0,60.0,9.0,75.0,67,West Englewood,29647,WEST ENGLEWOOD,6.324417,5.059534,0.758930,0.505953,0.000000,5.059534,60,0.150000,5.059534,60
23,ENGLEWOOD,0.0,4.0,58.0,7.0,69.0,68,Englewood,24369,ENGLEWOOD,7.078666,5.950183,0.718125,0.410357,0.000000,9.540810,93,0.075269,9.540810,93
27,GARFIELD RIDGE,0.0,3.0,2.0,86.0,91.0,56,Garfield Ridge,35439,GARFIELD RIDGE,6.419481,0.141088,6.066763,0.211631,0.000000,11.428088,162,0.530864,11.428088,162
75,WOODLAWN,2.0,2.0,0.0,0.0,4.0,42,Woodlawn,24425,WOODLAWN,0.409417,0.000000,0.000000,0.204708,0.204708,12.282497,120,0.000000,12.282497,120
8,BELMONT CRAGIN,0.0,5.0,8.0,54.0,67.0,19,Belmont Cragin,78116,BELMONT CRAGIN,2.144247,0.256029,1.728199,0.160018,0.000000,8.929029,279,0.193548,8.929029,279


In [53]:
# number of police dockets by community, since there's a small number
df_community_summary.sort_values(by='count_POLICE',ascending = False).head()

,community,count_BAFCONP,count_POLICE,count_STRTSAN,count_TRANPORT,n_dockets,GEOID,COMMUNITY_NAME,2020_POP,COMMUNITY_CAPS,dp10k,streets_p10k,cdot_p10k,police_p10k,business_p10k,Sidewalks Per 10k_x,snow_complaints,citation_rate,Sidewalks Per 10k_y,snow_complaints
69,WEST ENGLEWOOD,0.0,6.0,60.0,9.0,75.0,67,West Englewood,29647,WEST ENGLEWOOD,6.324417,5.059534,0.758930,0.505953,0.000000,5.059534,60,0.150000,5.059534,60
8,BELMONT CRAGIN,0.0,5.0,8.0,54.0,67.0,19,Belmont Cragin,78116,BELMONT CRAGIN,2.144247,0.256029,1.728199,0.160018,0.000000,8.929029,279,0.193548,8.929029,279
23,ENGLEWOOD,0.0,4.0,58.0,7.0,69.0,68,Englewood,24369,ENGLEWOOD,7.078666,5.950183,0.718125,0.410357,0.000000,9.540810,93,0.075269,9.540810,93
27,GARFIELD RIDGE,0.0,3.0,2.0,86.0,91.0,56,Garfield Ridge,35439,GARFIELD RIDGE,6.419481,0.141088,6.066763,0.211631,0.000000,11.428088,162,0.530864,11.428088,162
75,WOODLAWN,2.0,2.0,0.0,0.0,4.0,42,Woodlawn,24425,WOODLAWN,0.409417,0.000000,0.000000,0.204708,0.204708,12.282497,120,0.000000,12.282497,120


### top communities for business affairs dockets per capita

In [54]:
df_community_summary.sort_values(by='business_p10k',ascending = False).head()

,community,count_BAFCONP,count_POLICE,count_STRTSAN,count_TRANPORT,n_dockets,GEOID,COMMUNITY_NAME,2020_POP,COMMUNITY_CAPS,dp10k,streets_p10k,cdot_p10k,police_p10k,business_p10k,Sidewalks Per 10k_x,snow_complaints,citation_rate,Sidewalks Per 10k_y,snow_complaints
75,WOODLAWN,2.0,2.0,0.0,0.0,4.0,42,Woodlawn,24425,WOODLAWN,0.409417,0.000000,0.000000,0.204708,0.204708,12.282497,120,0.000000,12.282497,120
48,NEAR WEST SIDE,0.0,0.0,3.0,83.0,86.0,28,Near West Side,67881,NEAR WEST SIDE,3.167307,0.110487,3.056820,0.000000,0.000000,21.729203,590,0.140678,21.729203,590
54,OAKLAND,0.0,0.0,1.0,12.0,13.0,36,Oakland,6799,OAKLAND,4.780115,0.367701,4.412414,0.000000,0.000000,15.443448,42,0.285714,15.443448,42
53,NORWOOD PARK,0.0,0.0,1.0,5.0,6.0,10,Norwood Park,38303,NORWOOD PARK,0.391614,0.065269,0.326345,0.000000,0.000000,11.422082,175,0.028571,11.422082,175
52,NORTH PARK,0.0,0.0,1.0,9.0,10.0,13,North Park,17559,NORTH PARK,1.423771,0.142377,1.281394,0.000000,0.000000,14.807221,104,0.086538,14.807221,104


# Export for Analysis

In [55]:
df_community_summary.to_csv("../../results/dockets-by-community.csv", index= False)